In [1]:
%reload_ext Cython
%reload_ext line_profiler

In [2]:
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

%matplotlib tk
import seaborn as sns
sns.set_context('notebook', font_scale=1.5,
                rc={'lines.markeredgewidth': 1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

/usr/lib64/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
from PyHEADTAIL.trackers.rf_bucket import RFBucket
import PyHEADTAIL.particles.generators as gen

PyHEADTAIL v1.9.3.13




### Create bunches and bunches_list

In [4]:
momentum = 450e9 * e/c
gamma = np.sqrt((momentum/(m_p*c))**2 + 1)
beta = np.sqrt(1 - gamma**-2)

circumference = 40
alpha = 3.225e-4
h1 = 20
V1 = 1000
p_increment = 0 * e/c * circumference/(beta*c)

print gamma

479.606062093


In [5]:
rfbucket = RFBucket(
    charge=e, mass=m_p, gamma=gamma,
    circumference=circumference,
    alpha_array=[alpha], p_increment=0,
    harmonic_list=[h1], voltage_list=[V1], phi_offset_list=[0])

In [6]:
dt = np.random.uniform(0, 10e-9, 3)
epsn_z = np.random.uniform(0, 0.5, 3)
print dt
print epsn_z

[  8.78011096e-09   8.25225359e-09   3.25650744e-09]
[ 0.32427764  0.04757782  0.43033839]


In [7]:
T0 = circumference/beta/c
dt = (0, 2.5e-9, 7.5e-9, T0+0, T0+2.5e-9, T0+7.5e-9)
epsn_z = (0.35, 0.25, 0.1, 0.35, 0.25, 0.1)
# epsn_z = (0.1, 0.25, 0.35, 0.1, 0.25, 0.35)

In [8]:
bunches_list = [
    gen.ParticleGenerator(
        macroparticlenumber=1e5, intensity=1e11,
        charge=e, mass=m_p, gamma=gamma,
        circumference=circumference,
        distribution_x=gen.gaussian2D(2e-6),
        distribution_y=gen.gaussian2D(2e-6),
        distribution_z=gen.RF_bucket_distribution(rfbucket, epsn_z=epsn_z[i])
    ).generate() for i in xrange(6)]

# bunches_list = [deepcopy(b) for i in range(2) for b in bunches_list]
for i, b in enumerate(bunches_list):
    b.z -= dt[i] * (b.beta*c)
    b.dt = dt[i]

bunches = sum(bunches_list)
bunches.dt = dt[0]

*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 2.50e-02
... distance to target emittance: 2.51e-02
... distance to target emittance: -1.77e-04
--> Emittance: 0.350001068509
--> Bunch length:0.164249527414

/home/kli/workspace/PyHEADTAIL/particles/generators.py:330: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  coords = [np.random.normal(loc=0., scale=std, size=n_particles),
/home/kli/workspace/PyHEADTAIL/particles/generators.py:331: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  np.random.normal(loc=0., scale=std, size=n_particles)]
/home/kli/workspace/PyHEADTAIL/particles/generators.py:530: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  u = uniform(low=xmin, high=xmax, size=n_gen)



*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 1.35e-02
... distance to target emittance: 1.36e-02
... distance to target emittance: 2.72e-05
--> Emittance: 0.250000058984
--> Bunch length:0.137176493154
*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 1.90e-03
... distance to target emittance: 2.02e-03
... distance to target emittance: 7.91e-07
--> Emittance: 0.100000000329
--> Bunch length:0.0853883539502
*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 2.50e-02
... distance to target emittance: 2.51e-02
... distance to target emittance: -1.77e-04
--> Emittance: 0.350001068509
--> Bunch length:0.164249527414
*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 1.35e-02
... distance to target emittance: 1.36e-02
... distance to target emittance: 2.72e-05
--> Emittance: 0.250000058984
--> Bunch length:0.137176493154
*** Maximum RMS emittance 1.12145002894eV s.
... dist

/home/kli/workspace/PyHEADTAIL/particles/generators.py:531: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  v = uniform(low=ymin, high=ymax, size=n_gen)
/home/kli/workspace/PyHEADTAIL/particles/generators.py:532: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  s = uniform(size=n_gen)


In [9]:
print bunches_list
print [b.mean_z() for b in bunches_list]
print [b.epsn_z() for b in bunches_list]
print [b.dt for b in bunches_list]

[<PyHEADTAIL.particles.particles.Particles object at 0x7f67f1f2ac50>, <PyHEADTAIL.particles.particles.Particles object at 0x7f67f1f2add0>, <PyHEADTAIL.particles.particles.Particles object at 0x7f67f1f2af90>, <PyHEADTAIL.particles.particles.Particles object at 0x7f67f1ec2190>, <PyHEADTAIL.particles.particles.Particles object at 0x7f67f1ec2350>, <PyHEADTAIL.particles.particles.Particles object at 0x7f67f1ec2510>]
[0.00065432033640587647, -0.74854430435652308, -2.2489821610391307, -40.000101823403391, -40.749698812625297, -42.248139343109045]
[0.3500811936608908, 0.25035558827137228, 0.099318083990680511, 0.34859437850085245, 0.25011768808613033, 0.099647057149103574]
[0, 2.5e-09, 7.5e-09, 1.3342592810823143e-07, 1.3592592810823143e-07, 1.4092592810823144e-07]


### Wakes and wake kicks and functions

In [10]:
from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.impedances.wakes import CircularResonator, WakeField

In [11]:
slicer = UniformBinSlicer(60, z_cuts=(-.3, .3))

wake = CircularResonator(R_shunt=1e9, frequency=1e9, Q=500, n_turns_wake=3)

wakefields = WakeField(slicer, wake, circumference=circumference)
wakekick = wakefields.wake_kicks[0]
wakefunction = wakekick.wake_function

*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!
*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


### Convolutions

In [12]:
def convolution_python(t_target, t_source, c_source, w):
    dxp = 0.*t_target
    for k in xrange(len(t_target)):
        for l in xrange(len(t_source)):
            dxp[k] += c_source[l]*w(t_target[k]-t_source[l])

    return dxp

In [13]:
def convolution_numpy(t_target, t_source, c_source, w):
    tmin, tmax = t_source[0], t_source[-1]
    tt = np.concatenate((t_target-tmax, (t_target-tmin)[1:]))
    
    return np.convolve(c_source, w(tt), mode='valid')

In [14]:
def convolve_multibunch(bunches, times, ages, cmoments, betas, wf, f_convolution=convolution_python):
    dxp = []
    bunches = np.atleast_1d(bunches)
    
    n_turns, n_bunches, n_slices = times.shape
    dt = [b.dt for b in bunches]

    for i in xrange(n_bunches):
        z = 0.*times[i]
        t_target = times[i]
        n_bunches_infront = i+1
        for k in xrange(n_turns):
            if k>0:
                n_bunches_infront = n_bunches
            for j in range(n_bunches_infront):
                t_source = times[k,j] - ages[k] + dt[i] - dt[j]
                c_source = cmoments[k,j]
                z += f_convolution(t_target, t_source, c_source, wf) # betas not used here thanks to wake
            
        dxp.append(z)
        
    return dxp

In [15]:
def convolve_PyHEADTAIL_sb(bunch, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal(bunch, times, ages, cmoments, betas)

In [16]:
def convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal_multibunch(bunches, times, ages, cmoments, betas)

### Use cases
#### Create slices and slices_list
---

In [17]:
def get_times_and_moments(bunches_list, slicer):
    bunches_list = np.atleast_1d(bunches_list)

    slices_list = []
    times_list = []
    cmoments_list = []
    for i, b in enumerate(bunches_list):
        distance = dt[i]*b.beta*c
        b.z += distance
        slices_list.append(b.get_slices(slicer))
        times_list.append(slices_list[-1].z_centers / (b.beta*c))
        cmoments_list.append(slices_list[-1].charge_per_slice)
        b.z -= distance

    # Multi-turn multi-bunch times and moments arrays - here, only one turn assumed
    n_turns = 1
    n_bunches = len(bunches_list)
    n_slices = len(times_list[0])

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = times_list
    times_array = tmp

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = cmoments_list
    cmoments_array = tmp

    ages_list = np.array([0 for b in bunches_list])
    betas_list = np.array([[b.beta for b in bunches_list]])
    
    return times_array, ages_list, cmoments_array, betas_list

#### Single beam

In [18]:
# Single beam
slicer = UniformBinSlicer(2000, z_cuts=(-45, .5))

times, ages, cmoments, betas = get_times_and_moments(bunches, slicer)

delta_xp_mb = convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)
delta_xp_np = convolve_multibunch(bunches, times, ages, cmoments, betas, wakefunction)
# %timeit convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)
# %timeit convolve_multibunch(times_list, cmoments_list, wakefunction)
# %timeit convolve_multibunch(times_list, cmoments_list, wakefunction, f_convolution=convolution_numpy)

times_list = times[:,0,:]
cmoments_list = cmoments[:,0,:]

delta_xp_sb = convolve_PyHEADTAIL_sb(bunches, times_list, ages, cmoments_list, betas)
# %timeit convolve_PyHEADTAIL_sb(bunches, times_list, ages, cmoments_list, betas)

In [19]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('husl', 6)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-11e-9, 3e-9)

ax2.plot(dxp1/max(dxp1))
ax2.plot(dxp2/max(dxp2))
ax2.plot(-dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'])

ax3.plot(times-times[-1], 2*cmoments/max(cmoments))
ax3.plot(times-times[-1], dxp1/max(dxp1), label="all bunches")

# ax3.plot(times_reg-times[-1], dxp_mt/max(dxp1), label="mt bunches")

ax3.legend()
ax3.set_xlim(-11e-9, 3e-9)
ax3.set_ylim(-2.1, 2.1)

(-2.1, 2.1)

#### Multi turn test
---

In [58]:
slicer = UniformBinSlicer(500, z_cuts=(-5, 0.5))

bunches_source = sum(bunches_list[:3])
bunches_target = sum(bunches_list[3:])
times_s, ages_s, cmoments_s, betas_s = get_times_and_moments(bunches_source, slicer)
times_t, ages_t, cmoments_t, betas_t = get_times_and_moments(bunches_source, slicer)

bunches_target.dt = 0

n_turns, n_bunches = 2, 1
times_reg = np.zeros((n_turns, n_bunches, slicer.n_slices))
cmoments_reg = np.zeros((n_turns, n_bunches, slicer.n_slices))

times_reg[0,0,:] = times_t
times_reg[1,0,:] = times_s
cmoments_reg[0,0,:] = cmoments_t
cmoments_reg[1,0,:] = cmoments_s

ages = [0., T0]
betas = [bunches_target.beta, bunches_source.beta]

In [60]:
def convolve_multibunch(bunches, times, ages, cmoments, betas, wf, f_convolution=convolution_python):
    dxp = []
    bunches = np.atleast_1d(bunches)
    
    n_turns, n_bunches, n_slices = times.shape
#     print n_turns, n_bunches, n_slices
    dt = [b.dt for b in bunches]

    for i in xrange(n_bunches):
        z = 0
        t_target = times[0,i]
        n_bunches_infront = i+1
        for k in xrange(n_turns):
            if k>0:
                n_bunches_infront = n_bunches
            for j in range(n_bunches_infront):
                t_source = times[k,j] + ages[k] + dt[i] - dt[j]
                print t_target[:1], t_source[:1], ages[k], n_bunches_infront, i, j, k
                c_source = cmoments[k,j]
                z += f_convolution(t_target, t_source, c_source, wf) # betas not used here thanks to wake
            
        dxp.append(z)
        
    return dxp

In [61]:
def convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal_multibunch(bunches, times, ages, cmoments, betas)

In [62]:
dxp_mt = convolve_PyHEADTAIL_mb(bunches_target, times_reg, ages, cmoments_reg, betas)
dxp_mt_np = convolve_multibunch(bunches, times_reg, ages, cmoments_reg, betas, wakefunction)
dxp_mt_np = np.squeeze(dxp_mt_np[0])

[ -1.66598949e-08] [ -1.66598949e-08] 0.0 1 0 0 0
[ -1.66598949e-08] [  1.16766033e-07] 1.33425928108e-07 1 0 0 1


In [64]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('husl', 6)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-11e-9, 3e-9)

ax2.plot(dxp1/max(dxp1))
ax2.plot(dxp2/max(dxp2))
ax2.plot(-dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'])

ax3.plot(times-times[-1], 2*cmoments/max(cmoments))
ax3.plot(times-times[-1], dxp1/max(dxp1), label="all bunches")

for i in range(n_turns):
    ax3.plot(times_reg[i,0,:]-times[-1]-ages[i], dxp_mt[0]/max(dxp_mt[0]))
    ax3.plot(times_reg[i,0,:]-times[-1]-ages[i], -dxp_mt_np/max(-dxp_mt_np))
    ax3.plot(times_reg[i,0,:]-times[-1]-ages[i], 2*cmoments_reg[i,0,:]/max(cmoments_reg[i,0,:]), c='y')

ax3.legend()
# ax3.set_xlim(-11e-9, 3e-9)
ax3.set_ylim(-2.1, 2.1)

(-2.1, 2.1)

#### Multiple bunches

In [ ]:
slicer = UniformBinSlicer(60, z_cuts=(-.3, .3))

times_list, ages, cmoments_list, betas = get_times_and_moments(bunches_list, slicer)

delta_xp_mb_l = convolve_PyHEADTAIL_mb(bunches_list, times_list, ages, cmoments_list, betas)
# %timeit convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)

delta_xp_sb_l = []
delta_xp_np_l = []
for i, b in enumerate(bunches_list):
    delta_xp_sb_l.append(convolve_PyHEADTAIL_sb(
            b, times_list[:,i,:], ages, cmoments_list[:,i,:], betas))
    delta_xp_np_l.append(convolve_multibunch(
            times_list[:,i,:], cmoments_list[:,i,:], wakefunction, f_convolution=convolution_numpy))

In [ ]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

dxp1_l = np.squeeze(delta_xp_mb_l)
dxp2_l = np.squeeze(delta_xp_sb_l)
dxp3_l = np.squeeze(delta_xp_np_l)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('husl', 7)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-11e-9, 3e-9)

ax2.plot(dxp1/max(dxp1))
ax2.plot(dxp2/max(dxp2))
ax2.plot(-dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'])

ax3.plot(times-times[-1], 2*cmoments/max(cmoments), '--')
ax3.plot(times-times[-1], dxp1/max(dxp1), '--', label="all bunches")

labels = ["first bunch", "second bunch", "third bunch"]
for i in xrange(len(bunches_list)):
    tml = np.squeeze(times_list[:,i,:])
    cml = np.squeeze(cmoments_list[:,i,:])
    ax3.plot(tml-times[-1]-dt[i], 2*cml/np.max(cmoments_list), c=col[i], label=labels[i])
    ax3.plot(tml-times[-1]-dt[i], dxp1_l[i]/max(dxp1), c=col[i+3])

#     [ax3.axvline(s, c=c[0], alpha=0.8) for s in (dts-dts[-1])]
#     [ax3.axvline(s, c=c[1], alpha=0.8) for s in (dts-dts[-1]-dt[i])]
#     [ax3.axvline(s, c=c[2], alpha=0.8) for s in (alltimes-alltimes[-1])]

ax3.legend()
ax3.set_xlim(-11e-9, 3e-9)
ax3.set_ylim(-2.1, 2.1)

In [ ]:
plt.close('all')